In [9]:
# IPL Match Winner Prediction System

# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 2: Load IPL Data
ipl_data = pd.read_csv('matches.csv')

# Step 3: Explore the Data
print(ipl_data.head())
print(ipl_data.columns)

# Step 4: Data Preprocessing
# Select relevant columns
ipl_data = ipl_data[['season', 'city', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner']]

# Drop rows with missing values
ipl_data.dropna(inplace=True)

# Clean and convert 'season' column to integer
ipl_data['season'] = ipl_data['season'].apply(lambda x: int(x.split('/')[0]))

# Encode categorical variables
team_encoder = LabelEncoder()
city_encoder = LabelEncoder()
venue_encoder = LabelEncoder()

# Encode all object-type columns
for col in ['team1', 'team2', 'toss_winner', 'winner']:
    ipl_data[col] = team_encoder.fit_transform(ipl_data[col])

ipl_data['city'] = city_encoder.fit_transform(ipl_data['city'])
ipl_data['venue'] = venue_encoder.fit_transform(ipl_data['venue'])

# Encode toss decision (bat = 0, field = 1)
ipl_data['toss_decision'] = ipl_data['toss_decision'].apply(lambda x: 0 if x == 'bat' else 1)

# Step 5: Feature Engineering
X = ipl_data[['season', 'city', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision']]
y = ipl_data['winner']

# Step 6: Split Data for Training and Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train the Model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Step 8: Evaluate Model
predictions = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, predictions) * 100:.2f}%")

# Step 9: Predict Match Winner
def predict_match_winner(team1_name, team2_name, toss_winner_name, toss_decision_name, city_name, venue_name, season_year):
    try:
        team1 = team_encoder.transform([team1_name])[0]
        team2 = team_encoder.transform([team2_name])[0]
        toss_winner = team_encoder.transform([toss_winner_name])[0]
        toss_decision = 0 if toss_decision_name == 'bat' else 1
        city = city_encoder.transform([city_name])[0]
        venue = venue_encoder.transform([venue_name])[0]

        match_features = [[season_year, city, venue, team1, team2, toss_winner, toss_decision]]

        predicted_winner = model.predict(match_features)[0]
        predicted_winner_name = team_encoder.inverse_transform([predicted_winner])[0]

        return f"🏆 Predicted Winner: {predicted_winner_name}"

    except Exception as e:
        return f"Error: {e}"

# Example Prediction
print(predict_match_winner('Mumbai Indians', 'Chennai Super Kings', 'Mumbai Indians', 'field', 'Mumbai', 'Wankhede Stadium', 2024))


       id   season        city        date match_type player_of_match  \
0  335982  2007/08   Bangalore  2008-04-18     League     BB McCullum   
1  335983  2007/08  Chandigarh  2008-04-19     League      MEK Hussey   
2  335984  2007/08       Delhi  2008-04-19     League     MF Maharoof   
3  335985  2007/08      Mumbai  2008-04-20     League      MV Boucher   
4  335986  2007/08     Kolkata  2008-04-20     League       DJ Hussey   

                                        venue                        team1  \
0                       M Chinnaswamy Stadium  Royal Challengers Bangalore   
1  Punjab Cricket Association Stadium, Mohali              Kings XI Punjab   
2                            Feroz Shah Kotla             Delhi Daredevils   
3                            Wankhede Stadium               Mumbai Indians   
4                                Eden Gardens        Kolkata Knight Riders   

                         team2                  toss_winner toss_decision  \
0        Kolkat

C:\Users\JUSTIN\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [54]:
import pandas as pd
import numpy as np
import requests
import json
import tkinter as tk
from tkinter import ttk, Frame, Scrollbar, Text
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from PIL import Image, ImageTk
from io import BytesIO
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('IPL_Predictor')

# Data Load karo
data = pd.read_csv('matches.csv')

# Relevant columns select karo
cols = ['team1', 'team2', 'toss_winner', 'winner', 'season', 'venue']
df = data[cols].dropna()

# Season ko integer mein convert karo
df['season'] = df['season'].apply(lambda x: int(str(x)[:4]))

# Home Advantage Feature Add karo
df['home_advantage'] = (df['team1'] == df['venue']).astype(int)

# Label Encoding for categorical variables
label_encoders = {}
for col in ['team1', 'team2', 'toss_winner', 'winner', 'venue']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Features aur Target
X = df[['team1', 'team2', 'toss_winner', 'season', 'home_advantage']]
y = df['winner']

# Hyperparameter Tuning ke liye Grid Search
param_grid = {
    'n_estimators': [300, 400],
    'max_depth': [8, 12],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0]
}

model = XGBClassifier(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

best_model = grid_search.best_estimator_

# Model Evaluation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

report = classification_report(y_test, y_pred, zero_division=0)
logger.info(f"Model average accuracy: {np.mean(accuracies):.4f}")

# Tkinter Window Create karo
root = tk.Tk()
root.title("IPL Match Predictor - Advanced Stats")
root.geometry("1200x800")
root.configure(bg="#f0f0f0")

# Create main frames
top_frame = Frame(root, bg="#f0f0f0")
top_frame.pack(fill="x", padx=10, pady=5)

prediction_frame = Frame(root, bg="#f0f0f0", relief="ridge", borderwidth=2)
prediction_frame.pack(fill="x", padx=10, pady=5)

graph_frame = Frame(root, bg="#f0f0f0")
graph_frame.pack(fill="x", padx=10, pady=5)

player_stats_frame = Frame(root, bg="#f0f0f0")
player_stats_frame.pack(fill="both", expand=True, padx=10, pady=5)

# Title and header
header_label = ttk.Label(top_frame, text="IPL Match Predictor & Live Stats", 
                        font=("Helvetica", 24, "bold"), background="#f0f0f0")
header_label.pack(pady=10)

# Status label for debugging/error messages
status_label = ttk.Label(top_frame, text="", font=("Helvetica", 10), foreground="gray", background="#f0f0f0")
status_label.pack(pady=2)

# Prediction section
prediction_label = ttk.Label(prediction_frame, text="🏏 Loading match data...", 
                           font=("Helvetica", 16, "bold"), background="#f0f0f0")
prediction_label.pack(pady=10)

match_details_label = ttk.Label(prediction_frame, text="", 
                             font=("Helvetica", 12), background="#f0f0f0")
match_details_label.pack(pady=5)

# Graph section
fig = Figure(figsize=(10, 4), dpi=100)
ax = fig.add_subplot(111)
canvas = FigureCanvasTkAgg(fig, master=graph_frame)
canvas.get_tk_widget().pack(fill="both", expand=True)

# Player stats section with tabs
notebook = ttk.Notebook(player_stats_frame)
notebook.pack(fill="both", expand=True)

batting_frame = Frame(notebook)
bowling_frame = Frame(notebook)
info_frame = Frame(notebook)

notebook.add(batting_frame, text="Batting Stats")
notebook.add(bowling_frame, text="Bowling Stats")
notebook.add(info_frame, text="Match Info")

# Batting stats text widget
batting_stats_text = Text(batting_frame, height=15, width=100, font=("Consolas", 11))
batting_stats_text.pack(fill="both", expand=True, padx=5, pady=5)
batting_scroll = Scrollbar(batting_frame, command=batting_stats_text.yview)
batting_scroll.pack(side="right", fill="y")
batting_stats_text.config(yscrollcommand=batting_scroll.set)

# Bowling stats text widget
bowling_stats_text = Text(bowling_frame, height=15, width=100, font=("Consolas", 11))
bowling_stats_text.pack(fill="both", expand=True, padx=5, pady=5)
bowling_scroll = Scrollbar(bowling_frame, command=bowling_stats_text.yview)
bowling_scroll.pack(side="right", fill="y")
bowling_stats_text.config(yscrollcommand=bowling_scroll.set)

# Match info text widget
match_info_text = Text(info_frame, height=15, width=100, font=("Consolas", 11))
match_info_text.pack(fill="both", expand=True, padx=5, pady=5)
info_scroll = Scrollbar(info_frame, command=match_info_text.yview)
info_scroll.pack(side="right", fill="y")
match_info_text.config(yscrollcommand=info_scroll.set)

# Alternative data fetching function
def fetch_alternative_stats(match_id):
    """Try to fetch stats from an alternative endpoint if primary fails"""
    try:
        api_key = "8b77578c-3fe3-4c6d-8950-913886aee5db"
        alt_url = f"https://api.cricapi.com/v1/match_info?apikey={api_key}&id={match_id}"
        response = requests.get(alt_url, timeout=10)
        data = response.json()
        return data
    except Exception as e:
        logger.error(f"Error fetching alternative stats: {e}")
        return {}

# Function to extract player data from different API formats
def extract_player_data(match_data):
    batting_stats = []
    bowling_stats = []
    
    # Try primary scorecard path
    if 'scorecard' in match_data:
        for inning in match_data.get('scorecard', []):
            if 'batting' in inning:
                batting_stats.extend(inning.get('batting', []))
            if 'bowling' in inning:
                bowling_stats.extend(inning.get('bowling', []))
    
    # Try alternative paths for batting stats
    if not batting_stats:
        # Check in score section
        for score in match_data.get('score', []):
            if 'batsman' in score:
                batting_stats.extend([{'batsman': b, 'r': 0, 'b': 0, '4s': 0, '6s': 0, 'sr': 0} for b in score.get('batsman', [])])
            
        # Check in teams section
        for team in match_data.get('teamInfo', []):
            if 'players' in team:
                for player in team.get('players', []):
                    if 'batting' in player:
                        batting_stats.append(player['batting'])
    
    # Try alternative paths for bowling stats
    if not bowling_stats:
        # Check in score section
        for score in match_data.get('score', []):
            if 'bowler' in score:
                bowling_stats.extend([{'bowler': b, 'o': 0, 'm': 0, 'r': 0, 'w': 0, 'eco': 0} for b in score.get('bowler', [])])
            
        # Check in teams section
        for team in match_data.get('teamInfo', []):
            if 'players' in team:
                for player in team.get('players', []):
                    if 'bowling' in player:
                        bowling_stats.append(player['bowling'])
    
    return batting_stats, bowling_stats

# Function to format batting stats with improved error handling
def format_batting_stats(batting_stats):
    if not batting_stats:
        return "No batting stats available yet.\n\nPossible reasons:\n- Match hasn't started batting\n- API data structure doesn't contain batting info yet\n- Data is being updated in real-time\n\nPlease use 'Refresh Now' to check for updates."
    
    header = f"{'Player':<25} {'Runs':<6} {'Balls':<6} {'4s':<4} {'6s':<4} {'SR':<8}\n"
    header += "-" * 60 + "\n"
    
    formatted_stats = header
    for player in batting_stats:
        # Handle different API data structures
        name = str(player.get('batsman', player.get('name', 'Unknown')))[:25]
        runs = str(player.get('r', player.get('runs', 0)))
        balls = str(player.get('b', player.get('balls', 0)))
        fours = str(player.get('4s', player.get('fours', 0)))
        sixes = str(player.get('6s', player.get('sixes', 0)))
        sr = str(player.get('sr', player.get('strikerate', 0)))
        
        # Only add batsmen who have faced at least one ball or if all data is missing
        if int(balls) > 0 or (runs == '0' and balls == '0' and fours == '0' and sixes == '0'):
            formatted_stats += f"{name:<25} {runs:<6} {balls:<6} {fours:<4} {sixes:<4} {sr:<8}\n"
    
    return formatted_stats

# Function to format bowling stats with improved error handling
def format_bowling_stats(bowling_stats):
    if not bowling_stats:
        return "No bowling stats available yet.\n\nPossible reasons:\n- Match hasn't started bowling\n- API data structure doesn't contain bowling info yet\n- Data is being updated in real-time\n\nPlease use 'Refresh Now' to check for updates."
    
    header = f"{'Bowler':<25} {'Overs':<6} {'Maidens':<8} {'Runs':<6} {'Wickets':<8} {'Econ':<6}\n"
    header += "-" * 70 + "\n"
    
    formatted_stats = header
    for player in bowling_stats:
        # Handle different API data structures
        name = str(player.get('bowler', player.get('name', 'Unknown')))[:25]
        overs = str(player.get('o', player.get('overs', 0)))
        maidens = str(player.get('m', player.get('maidens', 0)))
        runs = str(player.get('r', player.get('runs', 0)))
        wickets = str(player.get('w', player.get('wickets', 0)))
        economy = str(player.get('eco', player.get('economy', 0)))
        
        # Only include bowlers who have bowled at least something
        if overs != '0' or runs != '0' or wickets != '0':
            formatted_stats += f"{name:<25} {overs:<6} {maidens:<8} {runs:<6} {wickets:<8} {economy:<6}\n"
    
    return formatted_stats

# Function to format match info
def format_match_info(match_data):
    info = "🏟️ MATCH INFORMATION\n"
    info += "=" * 50 + "\n\n"
    
    # Basic match details
    info += f"Match: {match_data.get('name', 'Unknown match')}\n"
    info += f"Venue: {match_data.get('venue', 'Unknown venue')}\n"
    info += f"Date: {match_data.get('date', 'Unknown date')}\n"
    info += f"Match Type: {match_data.get('matchType', 'T20')}\n\n"
    
    # Toss information
    toss_winner = match_data.get('tossWinner', 'Unknown')
    toss_choice = match_data.get('tossChoice', 'Unknown')
    info += f"Toss: {toss_winner} won the toss and chose to {toss_choice}\n\n"
    
    # Current status
    info += f"Match Status: {match_data.get('status', 'Unknown')}\n\n"
    
    # Team information
    teams = match_data.get('teams', [])
    if teams:
        info += "Teams:\n"
        for team in teams:
            info += f"  • {team}\n"
    
    # Recent events
    events = match_data.get('recentBallCommentary', [])
    if events:
        info += "\nRecent Ball Commentary:\n"
        for event in events[:5]:  # Show last 5 ball events
            info += f"  • {event.get('ballNumber', '')}: {event.get('commentary', '')}\n"
    
    # Add model info
    info += "\n\nModel Information:\n"
    info += f"Model Average Accuracy: {np.mean(accuracies):.2%}\n"
    info += "Features Used: Teams, Toss Winner, Season, Home Advantage\n"
    info += "Model Type: XGBoost Classifier\n"
    
    return info

# Prediction aur Live Data Update Function
def update_prediction():
    try:
        status_label.config(text="Fetching latest match data...")
        api_key = "8b77578c-3fe3-4c6d-8950-913886aee5db"
        url = f"https://api.cricapi.com/v1/currentMatches?apikey={api_key}&offset=0"
        response = requests.get(url, timeout=15)
        data = response.json()

        match_found = False
        match_id = None
        
        for match in data.get('data', []):
            # You can set specific match ID or look for IPL matches
            if 'IPL' in match.get('name', '') or match.get('id') == 'cacf2d34-41b8-41dd-91ed-5183d880084c':
                match_found = True
                match_id = match.get('id')
                team1 = match.get('teams', ['Team 1'])[0]
                team2 = match.get('teams', ['Team 2'])[1] if len(match.get('teams', [])) > 1 else 'Team 2'
                toss_winner = match.get('tossWinner', team1)
                venue = match.get('venue', '')
                season = 2024

                # Match ka current status update karo
                match_status = match.get('status', 'No live status available')

                def get_score(index):
                    try:
                        score_data = match.get('score', [{}])[index]
                        return score_data.get('r', 0), score_data.get('w', 0), score_data.get('o', 0), score_data.get('inning', '')
                    except IndexError:
                        return 0, 0, 0, ''

                team1_score, team1_wickets, team1_overs, team1_inning = get_score(0)
                team2_score, team2_wickets, team2_overs, team2_inning = get_score(1)

                # Update match details label
                match_details = f"🏏 {team1} vs {team2} at {venue}\n"
                match_details += f"📊 Status: {match_status}\n\n"
                match_details += f"🏏 {team1}: {team1_score}/{team1_wickets} in {team1_overs} overs\n"
                match_details += f"🏏 {team2}: {team2_score}/{team2_wickets} in {team2_overs} overs"
                
                match_details_label.config(text=match_details)

                # Encode safely for unseen categories
                def safe_encode(value, encoder, default=0):
                    return encoder.transform([value])[0] if value in encoder.classes_ else default

                # Try to encode values or use defaults
                try:
                    team1_encoded = safe_encode(team1, label_encoders['team1'])
                    team2_encoded = safe_encode(team2, label_encoders['team2'])
                    toss_winner_encoded = safe_encode(toss_winner, label_encoders['toss_winner'])
                    venue_encoded = safe_encode(venue, label_encoders['venue'])
                except Exception as e:
                    logger.error(f"Encoding error: {e}")
                    team1_encoded, team2_encoded, toss_winner_encoded, venue_encoded = 0, 1, 0, 0

                home_advantage = 1 if team1 == venue else 0

                features = pd.DataFrame([[team1_encoded, team2_encoded, toss_winner_encoded, season, home_advantage]], 
                                       columns=X.columns)

                # Get prediction with better error handling for probabilities
                try:
                    raw_probabilities = best_model.predict_proba(features)[0]
                    prediction = best_model.predict(features)[0]
                    
                    # Ensure probabilities are valid (between 0.01 and 0.99)
                    adjusted_probabilities = [max(0.01, min(p, 0.99)) for p in raw_probabilities]
                    
                    # Convert to team-specific probabilities
                    if len(raw_probabilities) >= 2:
                        # If we have multi-class probabilities, select the ones for our teams
                        team1_prob = adjusted_probabilities[team1_encoded] if team1_encoded < len(adjusted_probabilities) else 0.6
                        team2_prob = adjusted_probabilities[team2_encoded] if team2_encoded < len(adjusted_probabilities) else 0.4
                    else:
                        # If we have binary probabilities
                        team1_prob = adjusted_probabilities[0]
                        team2_prob = adjusted_probabilities[1] if len(adjusted_probabilities) > 1 else 1 - team1_prob
                    
                    # Ensure probabilities are complementary and sum to 1
                    total = team1_prob + team2_prob
                    if total > 0:
                        team1_prob = team1_prob / total
                        team2_prob = team2_prob / total
                    else:
                        team1_prob, team2_prob = 0.5, 0.5
                    
                    # Safely get predicted winner name
                    try:
                        predicted_winner = label_encoders['winner'].inverse_transform([prediction])[0]
                    except:
                        predicted_winner = team1 if prediction == team1_encoded else team2
                    
                    # Make sure predicted_winner is one of our two teams
                    if predicted_winner not in [team1, team2]:
                        predicted_winner = team1 if team1_prob > team2_prob else team2
                    
                    # Update prediction label
                    prediction_label.config(text=f"🏆 Predicted Winner: {predicted_winner}")
                    
                    # Store actual probabilities for plotting
                    team_probs = [team1_prob, team2_prob]
                    team_names = [team1, team2]
                    
                except Exception as e:
                    logger.error(f"Prediction error: {e}")
                    prediction_label.config(text="⚠️ Could not generate prediction")
                    team_probs = [0.5, 0.5]
                    team_names = [team1, team2]

                # Graph bana ke display karo with improved visualization
                ax.clear()
                colors = ['#1E90FF', '#FF6347']
                bars = ax.bar(team_names, team_probs, color=colors)
                ax.set_ylim(0, 1)
                ax.set_title("Win Probability", fontsize=16, fontweight='bold')
                ax.set_ylabel("Probability", fontsize=14)
                
                # Add percentage labels on top of bars
                for i, (bar, prob) in enumerate(zip(bars, team_probs)):
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                           f"{prob:.1%}", ha='center', va='bottom', fontsize=14, fontweight='bold')
                
                # Add grid for better readability
                ax.grid(axis='y', linestyle='--', alpha=0.7)
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                
                canvas.draw()

                # Get player stats with more robust extraction
                batting_stats, bowling_stats = extract_player_data(match)
                
                # If we couldn't find stats, try alternative API endpoint
                if not batting_stats and match_id:
                    status_label.config(text="Trying alternative data source for player stats...")
                    alt_data = fetch_alternative_stats(match_id)
                    alt_batting, alt_bowling = extract_player_data(alt_data)
                    
                    if alt_batting:
                        batting_stats = alt_batting
                    if alt_bowling:
                        bowling_stats = alt_bowling
                
                # Log the data we found
                logger.info(f"Found {len(batting_stats)} batting and {len(bowling_stats)} bowling records")
                
                # Update batting stats
                batting_stats_text.delete(1.0, tk.END)
                batting_stats_text.insert(tk.END, format_batting_stats(batting_stats))
                
                # Update bowling stats
                bowling_stats_text.delete(1.0, tk.END)
                bowling_stats_text.insert(tk.END, format_bowling_stats(bowling_stats))
                
                # Update match info
                match_info_text.delete(1.0, tk.END)
                match_info_text.insert(tk.END, format_match_info(match))
                
                status_label.config(text=f"Last updated: {pd.Timestamp.now().strftime('%I:%M:%S %p')}")
                break  # First matching game found
                
        if not match_found:
            prediction_label.config(text="⚠️ No IPL matches found in the API data")
            match_details_label.config(text="Please check back during live IPL matches")
            status_label.config(text="No matches found. Will retry in 60 seconds.")
    
    except Exception as e:
        error_msg = str(e)
        logger.error(f"Error updating data: {error_msg}")
        prediction_label.config(text=f"⚠️ Error updating data")
        status_label.config(text=f"Error: {error_msg[:50]}... Will retry in 60 seconds.")
    
    # 60 seconds mein refresh karo
    root.after(60000, update_prediction)  

# Add refresh button with indicator
def refresh_with_indicator():
    refresh_button.config(text="Refreshing...", state=tk.DISABLED)
    update_prediction()
    root.after(3000, lambda: refresh_button.config(text="Refresh Now", state=tk.NORMAL))

# Initial update call
refresh_button = ttk.Button(top_frame, text="Refresh Now", command=refresh_with_indicator)
refresh_button.pack(pady=5)

# Start with initial update
root.after(1000, update_prediction)

# Start the main loop
root.mainloop()

Fitting 5 folds for each of 16 candidates, totalling 80 fits


2025-03-22 20:23:24,693 - INFO - Model average accuracy: 0.5073
2025-03-22 20:23:28,711 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:24:04,308 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:24:31,484 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:25:06,930 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:25:21,789 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:25:34,006 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:26:09,434 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:26:24,877 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:26:36,470 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:26:48,907 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:27:12,046 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:27:27,391 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20:27:40,049 - INFO - Found 0 batting and 0 bowling records
2025-03-22 20